## 1. 환경 설정

`(1) Env 환경변수`

In [34]:
from dotenv import load_dotenv
load_dotenv()

True

`(2) 라이브러리`

In [35]:
import re
import os, json

from textwrap import dedent
from pprint import pprint

import warnings
warnings.filterwarnings("ignore")

## 2. 도구 호출 (Tool Calling)
- 도구 호출은 LLM이 특정 작업을 수행하기 위해 외부 기능을 호출하는 기능
- 이를 통해 LLM은 외부 API 통합 등 더 복잡한 작업을 수행할 수 있음 

### 2-1. 랭체인 내장 도구
- Tavily 웹 검색 도구 (예시)

`(1) 도구(tool) 정의하기`

In [36]:
from langchain_community.tools import TavilySearchResults

# 검색할 쿼리 설정
query = "스테이크와 어울리는 와인을 추천해주세요."

# Tavily 검색 도구 초기화 (최대 2개의 결과 반환)
web_search = TavilySearchResults(max_results=2)

# 웹 검색 실행
search_results = web_search.invoke(query)

# 검색 결과 출력
for result in search_results:
    print(result)  
    print("-" * 100)  

{'url': 'https://mashija.com/%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%99%80-%EC%96%B4%EC%9A%B8%EB%A6%AC%EB%8A%94-%EC%B5%9C%EA%B3%A0%EC%9D%98-%EC%99%80%EC%9D%B8-%EB%AC%B4%EC%97%87%EC%9D%84-%EA%B3%A0%EB%A5%BC-%EA%B2%83%EC%9D%B8/', 'content': '# 스테이크와 어울리는 최고의 와인: 무엇을 고를 것인가?\n\n카베르네 소비뇽(Cabernet Sauvignon) 및 말벡(Malbec)과 같은 전형적인 선택부터 더 가벼운 레드 와인, 심지어 화이트 와인과 맛있는 스테이크를 페어링하는 방법까지, 우리의 아카이브에서 가져온 최고의 조언과 최근 디캔터 전문가가 추천한 와인을 소개한다.\n\n<스테이크를 곁들인 레드 와인을 위한 5가지 전형적인 선택>\n\n• 카베르네 소비뇽(Cabernet Sauvignon)  \n• 말벡(Malbec)  \n• 그르나슈/쉬라즈 블렌드(Grenache / Shiraz blends)  \n• 시라/쉬라즈(Syrah / Shiraz)  \n• 산지오베제(Sangiovese)\n\n육즙이 풍부한 스테이크와 맛있는 와인이 있는 저녁 식사는 적어도 고기 애호가들에게 인생의 큰 즐거움일 것이다.'}
----------------------------------------------------------------------------------------------------
{'url': 'https://alcohol.hobby-tech.com/entry/%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%99%80-%EC%B0%B0%EB%96%A1%EA%B6%81%ED%95%A9-%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%97%90-%EC%96%B4%EC%9A%B8%EB%A6%AC%EB%8A%94-%EC%99%80%

In [37]:
# 도구 속성
print("자료형: ")
print(type(web_search))
print("-"*100)

print("name: ")
print(web_search.name)
print("-"*100)

print("description: ")
pprint(web_search.description)
print("-"*100)

print("schema: ")
pprint(web_search.args_schema.schema())
print("-"*100)

자료형: 
<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>
----------------------------------------------------------------------------------------------------
name: 
tavily_search_results_json
----------------------------------------------------------------------------------------------------
description: 
('A search engine optimized for comprehensive, accurate, and trusted results. '
 'Useful for when you need to answer questions about current events. Input '
 'should be a search query.')
----------------------------------------------------------------------------------------------------
schema: 
{'description': 'Input for the Tavily tool.',
 'properties': {'query': {'description': 'search query to look up',
                          'title': 'Query',
                          'type': 'string'}},
 'required': ['query'],
 'title': 'TavilyInput',
 'type': 'object'}
----------------------------------------------------------------------------------------------------

`(2) 도구(tool) 호출하기`

In [38]:
from langchain_openai import ChatOpenAI

# ChatOpenAI 모델 초기화
llm = ChatOpenAI(model="gpt-4o-mini")

# 웹 검색 도구를 직접 LLM에 바인딩 가능
llm_with_tools = llm.bind_tools(tools=[web_search])

In [39]:
# 도구 호출이 필요 없는 LLM 호출을 수행
query = "안녕하세요."
ai_msg = llm_with_tools.invoke(query)

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

AIMessage(content='안녕하세요! 어떻게 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 82, 'total_tokens': 93, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'finish_reason': 'stop', 'logprobs': None}, id='run--a386974b-82a8-484e-b85b-d5ed6f22e292-0', usage_metadata={'input_tokens': 82, 'output_tokens': 11, 'total_tokens': 93})
----------------------------------------------------------------------------------------------------
'안녕하세요! 어떻게 도와드릴까요?'
----------------------------------------------------------------------------------------------------
[]
----------------------------------------------------------------------------------------------------


In [40]:
# 도구 호출이 필요한 LLM 호출을 수행
query = "스테이크와 어울리는 와인을 추천해주세요."
ai_msg = llm_with_tools.invoke(query)

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_nPcsfpARCIM1JHjOqEE7lEgk', 'function': {'arguments': '{"query":"스테이크와 어울리는 와인 추천"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 91, 'total_tokens': 118, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--e2a76ef3-6edd-4448-b099-011c7ab0cd80-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '스테이크와 어울리는 와인 추천'}, 'id': 'call_nPcsfpARCIM1JHjOqEE7lEgk', 'type': 'tool_call'}], usage_metadata={'input_tokens': 91, 'output_tokens': 27, 'total_tokens': 118})
-------------------------------------------------

In [41]:
tool_call = ai_msg.tool_calls[0]
tool_call

{'name': 'tavily_search_results_json',
 'args': {'query': '스테이크와 어울리는 와인 추천'},
 'id': 'call_nPcsfpARCIM1JHjOqEE7lEgk',
 'type': 'tool_call'}

`(3) 도구(tool) 실행하기`

In [42]:
### 방법 1: 직접 도구 호출 처리

# 이 방법은 AI 메시지에서 첫 번째 도구 호출을 가져와 직접 처리한다.
# 'args'를 사용하여 도구를 호출하고 결과를 얻는다.

tool_output = web_search.invoke(tool_call["args"])
print(f"{tool_call['name']} 호출 결과:")
print("-" * 100)
print(tool_output)

tavily_search_results_json 호출 결과:
----------------------------------------------------------------------------------------------------
[{'url': 'https://mashija.com/%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%99%80-%EC%96%B4%EC%9A%B8%EB%A6%AC%EB%8A%94-%EC%B5%9C%EA%B3%A0%EC%9D%98-%EC%99%80%EC%9D%B8-%EB%AC%B4%EC%97%87%EC%9D%84-%EA%B3%A0%EB%A5%BC-%EA%B2%83%EC%9D%B8/', 'content': '# 스테이크와 어울리는 최고의 와인: 무엇을 고를 것인가?\n\n카베르네 소비뇽(Cabernet Sauvignon) 및 말벡(Malbec)과 같은 전형적인 선택부터 더 가벼운 레드 와인, 심지어 화이트 와인과 맛있는 스테이크를 페어링하는 방법까지, 우리의 아카이브에서 가져온 최고의 조언과 최근 디캔터 전문가가 추천한 와인을 소개한다.\n\n<스테이크를 곁들인 레드 와인을 위한 5가지 전형적인 선택>\n\n• 카베르네 소비뇽(Cabernet Sauvignon)  \n• 말벡(Malbec)  \n• 그르나슈/쉬라즈 블렌드(Grenache / Shiraz blends)  \n• 시라/쉬라즈(Syrah / Shiraz)  \n• 산지오베제(Sangiovese)\n\n육즙이 풍부한 스테이크와 맛있는 와인이 있는 저녁 식사는 적어도 고기 애호가들에게 인생의 큰 즐거움일 것이다.'}, {'url': 'https://alcohol.hobby-tech.com/entry/%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%99%80-%EC%B0%B0%EB%96%A1%EA%B6%81%ED%95%A9-%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%97%90-%EC%96%B4%E

In [43]:
### 방법 2: ToolMessage 객체 생성

# 이 방법은 도구 호출 결과를 사용하여 ToolMessage 객체를 생성한다.
# 도구 호출의 ID와 이름을 포함하여 더 구조화된 메시지를 만든다.

from langchain_core.messages import ToolMessage
tool_message = ToolMessage(
    content=tool_output,
    tool_call_id=tool_call["id"],
    name=tool_call["name"]
)

print(tool_message)

content=[{'url': 'https://mashija.com/%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%99%80-%EC%96%B4%EC%9A%B8%EB%A6%AC%EB%8A%94-%EC%B5%9C%EA%B3%A0%EC%9D%98-%EC%99%80%EC%9D%B8-%EB%AC%B4%EC%97%87%EC%9D%84-%EA%B3%A0%EB%A5%BC-%EA%B2%83%EC%9D%B8/', 'content': '# 스테이크와 어울리는 최고의 와인: 무엇을 고를 것인가?\n\n카베르네 소비뇽(Cabernet Sauvignon) 및 말벡(Malbec)과 같은 전형적인 선택부터 더 가벼운 레드 와인, 심지어 화이트 와인과 맛있는 스테이크를 페어링하는 방법까지, 우리의 아카이브에서 가져온 최고의 조언과 최근 디캔터 전문가가 추천한 와인을 소개한다.\n\n<스테이크를 곁들인 레드 와인을 위한 5가지 전형적인 선택>\n\n• 카베르네 소비뇽(Cabernet Sauvignon)  \n• 말벡(Malbec)  \n• 그르나슈/쉬라즈 블렌드(Grenache / Shiraz blends)  \n• 시라/쉬라즈(Syrah / Shiraz)  \n• 산지오베제(Sangiovese)\n\n육즙이 풍부한 스테이크와 맛있는 와인이 있는 저녁 식사는 적어도 고기 애호가들에게 인생의 큰 즐거움일 것이다.'}, {'url': 'https://alcohol.hobby-tech.com/entry/%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%99%80-%EC%B0%B0%EB%96%A1%EA%B6%81%ED%95%A9-%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%97%90-%EC%96%B4%EC%9A%B8%EB%A6%AC%EB%8A%94-%EC%99%80%EC%9D%B8-5%EA%B0%80%EC%A7%80', 'content': '• 4. 메를로 (Merlot) – 부드럽고 균형 잡힌 와인\n\n• 5. 진판델 (Z

In [44]:
### 방법 3: 도구 직접 호출하여 바로 ToolMessage 객체 생성

# 이 방법은 도구를 직접 호출하여 ToolMessage 객체를 생성한다.
# 가장 간단하고 직관적인 방법으로, LangChain의 추상화를 활용한다.

tool_message = web_search.invoke(tool_call)

print(tool_message)


content='[{"url": "https://mashija.com/%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%99%80-%EC%96%B4%EC%9A%B8%EB%A6%AC%EB%8A%94-%EC%B5%9C%EA%B3%A0%EC%9D%98-%EC%99%80%EC%9D%B8-%EB%AC%B4%EC%97%87%EC%9D%84-%EA%B3%A0%EB%A5%BC-%EA%B2%83%EC%9D%B8/", "content": "# 스테이크와 어울리는 최고의 와인: 무엇을 고를 것인가?\\n\\n카베르네 소비뇽(Cabernet Sauvignon) 및 말벡(Malbec)과 같은 전형적인 선택부터 더 가벼운 레드 와인, 심지어 화이트 와인과 맛있는 스테이크를 페어링하는 방법까지, 우리의 아카이브에서 가져온 최고의 조언과 최근 디캔터 전문가가 추천한 와인을 소개한다.\\n\\n<스테이크를 곁들인 레드 와인을 위한 5가지 전형적인 선택>\\n\\n• 카베르네 소비뇽(Cabernet Sauvignon)  \\n• 말벡(Malbec)  \\n• 그르나슈/쉬라즈 블렌드(Grenache / Shiraz blends)  \\n• 시라/쉬라즈(Syrah / Shiraz)  \\n• 산지오베제(Sangiovese)\\n\\n육즙이 풍부한 스테이크와 맛있는 와인이 있는 저녁 식사는 적어도 고기 애호가들에게 인생의 큰 즐거움일 것이다."}, {"url": "https://alcohol.hobby-tech.com/entry/%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%99%80-%EC%B0%B0%EB%96%A1%EA%B6%81%ED%95%A9-%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%97%90-%EC%96%B4%EC%9A%B8%EB%A6%AC%EB%8A%94-%EC%99%80%EC%9D%B8-5%EA%B0%80%EC%A7%80", "content": "• 4. 메를로 (Merlot) – 부드럽고 균형 잡힌 와인\\

In [45]:
pprint(tool_message.tool_call_id)

'call_nPcsfpARCIM1JHjOqEE7lEgk'


In [46]:
pprint(tool_message.name)

'tavily_search_results_json'


In [47]:
pprint(tool_message.content)

('[{"url": '
 '"https://mashija.com/%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%99%80-%EC%96%B4%EC%9A%B8%EB%A6%AC%EB%8A%94-%EC%B5%9C%EA%B3%A0%EC%9D%98-%EC%99%80%EC%9D%B8-%EB%AC%B4%EC%97%87%EC%9D%84-%EA%B3%A0%EB%A5%BC-%EA%B2%83%EC%9D%B8/", '
 '"content": "# 스테이크와 어울리는 최고의 와인: 무엇을 고를 것인가?\\n\\n카베르네 소비뇽(Cabernet '
 'Sauvignon) 및 말벡(Malbec)과 같은 전형적인 선택부터 더 가벼운 레드 와인, 심지어 화이트 와인과 맛있는 스테이크를 '
 '페어링하는 방법까지, 우리의 아카이브에서 가져온 최고의 조언과 최근 디캔터 전문가가 추천한 와인을 소개한다.\\n\\n<스테이크를 곁들인 '
 '레드 와인을 위한 5가지 전형적인 선택>\\n\\n• 카베르네 소비뇽(Cabernet Sauvignon)  \\n• 말벡(Malbec)  '
 '\\n• 그르나슈/쉬라즈 블렌드(Grenache / Shiraz blends)  \\n• 시라/쉬라즈(Syrah / Shiraz)  '
 '\\n• 산지오베제(Sangiovese)\\n\\n육즙이 풍부한 스테이크와 맛있는 와인이 있는 저녁 식사는 적어도 고기 애호가들에게 '
 '인생의 큰 즐거움일 것이다."}, {"url": '
 '"https://alcohol.hobby-tech.com/entry/%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%99%80-%EC%B0%B0%EB%96%A1%EA%B6%81%ED%95%A9-%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%97%90-%EC%96%B4%EC%9A%B8%EB%A6%AC%EB%8A%94-%EC%99%80%EC%9D%B8-5%EA%B0%80%EC%A7%80", '
 '"content": "• 

In [48]:
ai_msg.tool_calls

[{'name': 'tavily_search_results_json',
  'args': {'query': '스테이크와 어울리는 와인 추천'},
  'id': 'call_nPcsfpARCIM1JHjOqEE7lEgk',
  'type': 'tool_call'}]

In [49]:
# batch 실행 - 도구 호출이 여러 개인 경우

# tool_messages = web_search.batch([tool_call])

tool_messages = web_search.batch(ai_msg.tool_calls)

print(tool_messages)
print("-" * 100)
pprint(tool_messages[0].content)

[ToolMessage(content='[{"url": "https://mashija.com/%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%99%80-%EC%96%B4%EC%9A%B8%EB%A6%AC%EB%8A%94-%EC%B5%9C%EA%B3%A0%EC%9D%98-%EC%99%80%EC%9D%B8-%EB%AC%B4%EC%97%87%EC%9D%84-%EA%B3%A0%EB%A5%BC-%EA%B2%83%EC%9D%B8/", "content": "# 스테이크와 어울리는 최고의 와인: 무엇을 고를 것인가?\\n\\n카베르네 소비뇽(Cabernet Sauvignon) 및 말벡(Malbec)과 같은 전형적인 선택부터 더 가벼운 레드 와인, 심지어 화이트 와인과 맛있는 스테이크를 페어링하는 방법까지, 우리의 아카이브에서 가져온 최고의 조언과 최근 디캔터 전문가가 추천한 와인을 소개한다.\\n\\n<스테이크를 곁들인 레드 와인을 위한 5가지 전형적인 선택>\\n\\n• 카베르네 소비뇽(Cabernet Sauvignon)  \\n• 말벡(Malbec)  \\n• 그르나슈/쉬라즈 블렌드(Grenache / Shiraz blends)  \\n• 시라/쉬라즈(Syrah / Shiraz)  \\n• 산지오베제(Sangiovese)\\n\\n육즙이 풍부한 스테이크와 맛있는 와인이 있는 저녁 식사는 적어도 고기 애호가들에게 인생의 큰 즐거움일 것이다."}, {"url": "https://alcohol.hobby-tech.com/entry/%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%99%80-%EC%B0%B0%EB%96%A1%EA%B6%81%ED%95%A9-%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%97%90-%EC%96%B4%EC%9A%B8%EB%A6%AC%EB%8A%94-%EC%99%80%EC%9D%B8-5%EA%B0%80%EC%A7%80", "content": "• 4. 메를로 (Merlot) – 부드

`(4) ToolMessage를 LLM에 전달하여 답변을 생성하기`

In [50]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿 
prompt = ChatPromptTemplate([
    ("system", f"You are a helpful AI assistant. Today's date is {today}."),
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# ChatOpenAI 모델 초기화 
llm = ChatOpenAI(model="gpt-4o-mini")

# LLM에 도구를 바인딩
llm_with_tools = llm.bind_tools(tools=[web_search])

# LLM 체인 생성
llm_chain = prompt | llm_with_tools

# 도구 실행 체인 정의
@chain
def web_search_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)
    print("ai_msg: \n", ai_msg)
    print("-"*100)
    tool_msgs = web_search.batch(ai_msg.tool_calls, config=config)
    print("tool_msgs: \n", tool_msgs)
    print("-"*100)
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)

# 체인 실행
response = web_search_chain.invoke("오늘 모엣샹동 샴페인의 가격은 얼마인가요?")

# 응답 출력 
pprint(response.content)

ai_msg: 
 content='' additional_kwargs={'tool_calls': [{'id': 'call_QMc7SXirMIuZlQjkUgzlnKs7', 'function': {'arguments': '{"query":"모엣샹동 샴페인 가격 2025년 11월"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 114, 'total_tokens': 149, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--e05a059c-add8-4e9b-80f1-98e4d4a438fa-0' tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '모엣샹동 샴페인 가격 2025년 11월'}, 'id': 'call_QMc7SXirMIuZlQjkUgzlnKs7', 'type': 'tool_call'}] usage_metadata={'input_tokens': 114, 'output_tokens': 35, 'total_tokens': 149}
-------------------------------------------

### 2-2. 사용자 정의 도구
- @tool decorator를 통해 사용자 정의 도구를 정의할 수 있음

`(1) 도구(tool) 정의하기`

In [51]:
from langchain_community.tools import TavilySearchResults
from langchain_core.tools import tool
from typing import List

# Tool 정의 
@tool
def search_web(query: str) -> str:
    """Searches the internet for information that does not exist in the database or for the latest information."""

    tavily_search = TavilySearchResults(max_results=2)
    docs = tavily_search.invoke(query)

    formatted_docs = "\n---\n".join([
        f'<Document href="{doc["url"]}"/>\n{doc["content"]}\n</Document>'
        for doc in docs
        ])

    if len(formatted_docs) > 0:
        return formatted_docs
    
    return "관련 정보를 찾을 수 없습니다."

In [52]:
# 도구 속성
print("자료형: ")
print(type(search_web))
print("-"*100)

print("name: ")
print(search_web.name)
print("-"*100)

print("description: ")
pprint(search_web.description)
print("-"*100)

print("schema: ")
pprint(search_web.args_schema.schema())
print("-"*100)

자료형: 
<class 'langchain_core.tools.structured.StructuredTool'>
----------------------------------------------------------------------------------------------------
name: 
search_web
----------------------------------------------------------------------------------------------------
description: 
('Searches the internet for information that does not exist in the database or '
 'for the latest information.')
----------------------------------------------------------------------------------------------------
schema: 
{'description': 'Searches the internet for information that does not exist in '
                'the database or for the latest information.',
 'properties': {'query': {'title': 'Query', 'type': 'string'}},
 'required': ['query'],
 'title': 'search_web',
 'type': 'object'}
----------------------------------------------------------------------------------------------------


In [53]:
query = "스테이크와 어울리는 와인을 추천해주세요."
search_result = search_web.invoke(query)

print(search_result)

<Document href="https://mashija.com/%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%99%80-%EC%96%B4%EC%9A%B8%EB%A6%AC%EB%8A%94-%EC%B5%9C%EA%B3%A0%EC%9D%98-%EC%99%80%EC%9D%B8-%EB%AC%B4%EC%97%87%EC%9D%84-%EA%B3%A0%EB%A5%BC-%EA%B2%83%EC%9D%B8/"/>
# 스테이크와 어울리는 최고의 와인: 무엇을 고를 것인가?

카베르네 소비뇽(Cabernet Sauvignon) 및 말벡(Malbec)과 같은 전형적인 선택부터 더 가벼운 레드 와인, 심지어 화이트 와인과 맛있는 스테이크를 페어링하는 방법까지, 우리의 아카이브에서 가져온 최고의 조언과 최근 디캔터 전문가가 추천한 와인을 소개한다.

<스테이크를 곁들인 레드 와인을 위한 5가지 전형적인 선택>

• 카베르네 소비뇽(Cabernet Sauvignon)  
• 말벡(Malbec)  
• 그르나슈/쉬라즈 블렌드(Grenache / Shiraz blends)  
• 시라/쉬라즈(Syrah / Shiraz)  
• 산지오베제(Sangiovese)

육즙이 풍부한 스테이크와 맛있는 와인이 있는 저녁 식사는 적어도 고기 애호가들에게 인생의 큰 즐거움일 것이다.
</Document>
---
<Document href="https://alcohol.hobby-tech.com/entry/%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%99%80-%EC%B0%B0%EB%96%A1%EA%B6%81%ED%95%A9-%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%97%90-%EC%96%B4%EC%9A%B8%EB%A6%AC%EB%8A%94-%EC%99%80%EC%9D%B8-5%EA%B0%80%EC%A7%80"/>
• 4. 메를로 (Merlot) – 부드럽고 균형 잡힌 와인

• 5. 진판델 (Zinfandel) – 달콤한 향과

In [54]:
# LLM에 도구를 바인딩
llm_with_tools = llm.bind_tools(tools=[search_web])

# 도구 호출이 필요한 LLM 호출을 수행
query = "스테이크와 어울리는 와인을 추천해주세요."
ai_msg = llm_with_tools.invoke(query)

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_BjZIBopCYBEfS7HWe2oGdOEt', 'function': {'arguments': '{"query": "스테이크와 어울리는 와인 추천"}', 'name': 'search_web'}, 'type': 'function'}, {'id': 'call_snfduWeZ3sVUZCMz7VMlT3eE', 'function': {'arguments': '{"query": "스테이크 와인 pairing"}', 'name': 'search_web'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 67, 'total_tokens': 125, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--ffc52759-6ce6-4cb5-8f4a-4d8be38d297e-0', tool_calls=[{'name': 'search_web', 'args': {'query': '스테이크와 어울리는 와인 추천'}, 'id': 'call_BjZIBopCYBEfS7HWe2oGdOEt', 'type': 'tool_call'}, {'name': 'search_w

`(2) LLM 도구 호출 성능 비교하기`

In [55]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_groq import ChatGroq

# 기본 LLM
llm_gemini_flash = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)
llm_gemini_pro = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0)
llm_groq = ChatGroq(model="llama3-70b-8192", temperature=0)

# LLM에 도구 바인딩하여 추가 
tools=[search_web]

gemini_flash_with_tools = llm_gemini_flash.bind_tools(tools)
gemini_pro_with_tools = llm_gemini_pro.bind_tools(tools)
groq_llama3_with_tools = llm_groq.bind_tools(tools)

Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring


- gemini-1.5-flash

In [56]:
# 도구 호출이 필요한 LLM 호출을 수행
query = "스테이크와 어울리는 와인을 추천해주세요."
# ai_msg = gemini_flash_with_tools.invoke(query)

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_BjZIBopCYBEfS7HWe2oGdOEt', 'function': {'arguments': '{"query": "스테이크와 어울리는 와인 추천"}', 'name': 'search_web'}, 'type': 'function'}, {'id': 'call_snfduWeZ3sVUZCMz7VMlT3eE', 'function': {'arguments': '{"query": "스테이크 와인 pairing"}', 'name': 'search_web'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 67, 'total_tokens': 125, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--ffc52759-6ce6-4cb5-8f4a-4d8be38d297e-0', tool_calls=[{'name': 'search_web', 'args': {'query': '스테이크와 어울리는 와인 추천'}, 'id': 'call_BjZIBopCYBEfS7HWe2oGdOEt', 'type': 'tool_call'}, {'name': 'search_w

- gemini-1.5-pro

In [57]:
# 도구 호출이 필요한 LLM 호출을 수행
query = "스테이크와 어울리는 와인을 추천해주세요."
# ai_msg = gemini_pro_with_tools.invoke(query)

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_BjZIBopCYBEfS7HWe2oGdOEt', 'function': {'arguments': '{"query": "스테이크와 어울리는 와인 추천"}', 'name': 'search_web'}, 'type': 'function'}, {'id': 'call_snfduWeZ3sVUZCMz7VMlT3eE', 'function': {'arguments': '{"query": "스테이크 와인 pairing"}', 'name': 'search_web'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 67, 'total_tokens': 125, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--ffc52759-6ce6-4cb5-8f4a-4d8be38d297e-0', tool_calls=[{'name': 'search_web', 'args': {'query': '스테이크와 어울리는 와인 추천'}, 'id': 'call_BjZIBopCYBEfS7HWe2oGdOEt', 'type': 'tool_call'}, {'name': 'search_w

- llama3-70b-8192

In [58]:
# 도구 호출이 필요한 LLM 호출을 수행
query = "스테이크와 어울리는 와인을 추천해주세요."
ai_msg = groq_llama3_with_tools.invoke(query)

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

BadRequestError: Error code: 400 - {'error': {'message': 'The model `llama3-70b-8192` has been decommissioned and is no longer supported. Please refer to https://console.groq.com/docs/deprecations for a recommendation on which model to use instead.', 'type': 'invalid_request_error', 'code': 'model_decommissioned'}}

### 2-3. Runnable 객체를 도구(tool) 변환
- 문자열이나 dict 입력을 받는 Runnable을 도구로 변환
- as_tool 메서드를 사용

`(1) Document Loader`

In [ ]:
from langchain_community.document_loaders import WikipediaLoader
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda
from pydantic import BaseModel, Field
from typing import List

# WikipediaLoader를 사용하여 위키피디아 문서를 검색하는 함수 
def search_wiki(input_data: dict) -> List[Document]:
    """Search Wikipedia documents based on user input (query) and return k documents"""
    query = input_data["query"]
    k = input_data.get("k", 2)  
    wiki_loader = WikipediaLoader(query=query, load_max_docs=k, lang="ko")
    wiki_docs = wiki_loader.load()
    return wiki_docs

# 도구 호출에 사용할 입력 스키마 정의 
class WikiSearchSchema(BaseModel):
    """Input schema for Wikipedia search."""
    query: str = Field(..., description="The query to search for in Wikipedia")
    k: int = Field(2, description="The number of documents to return (default is 2)")

# RunnableLambda 함수를 사용하여 위키피디아 문서 로더를 Runnable로 변환 
runnable = RunnableLambda(search_wiki)
wiki_search = runnable.as_tool(
    name="wiki_search",
    description=dedent("""
        Use this tool when you need to search for information on Wikipedia.
        It searches for Wikipedia articles related to the user's query and returns
        a specified number of documents. This tool is useful when general knowledge
        or background information is required.
    """),
    args_schema=WikiSearchSchema
)

In [ ]:
# 도구 속성
print("자료형: ")
print(type(wiki_search))
print("-"*100)

print("name: ")
print(wiki_search.name)
print("-"*100)

print("description: ")
pprint(wiki_search.description)
print("-"*100)

print("schema: ")
pprint(wiki_search.args_schema.schema())
print("-"*100)

자료형: 
<class 'langchain_core.tools.structured.StructuredTool'>
----------------------------------------------------------------------------------------------------
name: 
wiki_search
----------------------------------------------------------------------------------------------------
description: 
('Use this tool when you need to search for information on Wikipedia.\n'
 "It searches for Wikipedia articles related to the user's query and returns\n"
 'a specified number of documents. This tool is useful when general knowledge\n'
 'or background information is required.')
----------------------------------------------------------------------------------------------------
schema: 
{'description': 'Input schema for Wikipedia search.',
 'properties': {'k': {'default': 2,
                      'description': 'The number of documents to return '
                                     '(default is 2)',
                      'title': 'K',
                      'type': 'integer'},
                'q

In [ ]:
# 위키 검색 실행
query = "파스타의 유래"
wiki_results = wiki_search.invoke({"query":query})

# 검색 결과 출력
for result in wiki_results:
    print(result)  
    print("-" * 100)  

page_content='오르조(이탈리아어: orzo) 또는 리소니(이탈리아어: risoni, 단수: risone 리소네[*])는 이탈리아의 파스타이다. "오르조"는 라틴어:hordeum에서 유래했으며 "보리"를 뜻한다. "리소니"는 "큰 쌀"이라는 뜻이다. 파스타의 일종으로 큰 쌀알의 모양을 하고 있으며 솔방울이나 잣보다는 좀 더 작다. 보통 라구 등 수프와 함께 먹는다. 원래는 보리로 만들었지만 요즘에는 박력분으로 만드는 것이 흔해졌다. 다른 이름으로는 kritharáki ("little barley") 혹은 manéstra(그리스 요리)로 부르며, lisān al-`uṣfūr ("명금의 혀")라고 아랍 요리에서 불린다. 오르조를 두고 이탈리아의 쌀이라고 부르기도 한다.
터키에서는 아르파 셰흐리예(arpa şehriye)로 불리며, 셰흐리예의 하나이다. 필라브를 만들거나 초르바(수프)에 넣어 먹는다.


== 같이 보기 ==
파스티나
셰흐리예


== 각주 ==' metadata={'title': '오르조', 'summary': '오르조(이탈리아어: orzo) 또는 리소니(이탈리아어: risoni, 단수: risone 리소네[*])는 이탈리아의 파스타이다. "오르조"는 라틴어:hordeum에서 유래했으며 "보리"를 뜻한다. "리소니"는 "큰 쌀"이라는 뜻이다. 파스타의 일종으로 큰 쌀알의 모양을 하고 있으며 솔방울이나 잣보다는 좀 더 작다. 보통 라구 등 수프와 함께 먹는다. 원래는 보리로 만들었지만 요즘에는 박력분으로 만드는 것이 흔해졌다. 다른 이름으로는 kritharáki ("little barley") 혹은 manéstra(그리스 요리)로 부르며, lisān al-`uṣfūr ("명금의 혀")라고 아랍 요리에서 불린다. 오르조를 두고 이탈리아의 쌀이라고 부르기도 한다.\n터키에서는 아르파 셰흐리예(arpa şehriye)로 불리며, 셰흐리예의 하나이다. 필라브를 만들거나 초르바(수프)에 넣어 먹는다.', 'source': 'https://ko.wik

In [ ]:
# LLM에 도구를 바인딩 (2개의 도구 바인딩)
llm_with_tools = llm.bind_tools(tools=[search_web, wiki_search])

# 도구 호출이 필요한 LLM 호출을 수행
query = "서울 강남의 유명한 파스타 맛집은 어디인가요? 그리고 파스타의 유래를 알려주세요. "
ai_msg = llm_with_tools.invoke(query)

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_lbv3QRvbOHWlNrwFwlmWET9u', 'function': {'arguments': '{"query": "서울 강남 파스타 맛집 추천"}', 'name': 'search_web'}, 'type': 'function'}, {'id': 'call_wbVN5565BEQ5jdQqfkxcMu0k', 'function': {'arguments': '{"query": "파스타의 유래"}', 'name': 'wiki_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 174, 'total_tokens': 230, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--8cee790e-c245-41c2-82e4-e02a83f8672a-0', tool_calls=[{'name': 'search_web', 'args': {'query': '서울 강남 파스타 맛집 추천'}, 'id': 'call_lbv3QRvbOHWlNrwFwlmWET9u', 'type': 'tool_call'}, {'name': 'wiki_search', 'a

`(2) LCEL 체인`
- 위키피디아 문서를 검색하고 내용을 요약하는 체인

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from langchain_community.document_loaders import WikipediaLoader

# WikipediaLoader를 사용하여 위키피디아 문서를 검색하고 텍스트로 반환하는 함수 
def wiki_search_and_summarize(input_data: dict):
    wiki_loader = WikipediaLoader(query=input_data["query"], load_max_docs=2, lang="ko")
    wiki_docs = wiki_loader.load()

    formatted_docs =[
        f'<Document source="{doc.metadata["source"]}"/>\n{doc.page_content}\n</Document>'
        for doc in wiki_docs
        ]
    
    return formatted_docs

# 요약 프롬프트 템플릿
summary_prompt = ChatPromptTemplate.from_template(
    "Summarize the following text in a concise manner:\n\n{context}\n\nSummary:"
)

# LLM 및 요약 체인 설정
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
summary_chain = (
    {"context": RunnableLambda(wiki_search_and_summarize)}
    | summary_prompt | llm | StrOutputParser() 
)

# 요약 테스트 
summarized_text = summary_chain.invoke({"query":"파스타의 유래"})
pprint(summarized_text)

('오르조(또는 리소니)는 이탈리아의 파스타로, 보리에서 유래한 이름을 가지고 있으며 큰 쌀알 모양이다. 주로 수프와 함께 먹으며, 현재는 '
 '박력분으로 만들어진다. 터키에서는 아르파 셰흐리예로 불리며, 필라브나 수프에 사용된다. 이탈리아 요리는 기원전 4세기부터 발전해왔으며, '
 '지역에 따라 다양한 특색을 지닌다. 북부는 쌀과 유제품을, 남부는 올리브와 해산물을 주로 사용한다. 이탈리아 요리는 간결함과 재료의 질을 '
 '중시하며, 2013년 CNN에서 세계 최고의 요리로 선정되었다.')


In [ ]:
# 도구 호출에 사용할 입력 스키마 정의 
class WikiSummarySchema(BaseModel):
    """Input schema for Wikipedia search."""
    query: str = Field(..., description="The query to search for in Wikipedia")

# as_tool 메소드를 사용하여 도구 객체로 변환
wiki_summary = summary_chain.as_tool(
    name="wiki_summary",
    description=dedent("""
        Use this tool when you need to search for information on Wikipedia.
        It searches for Wikipedia articles related to the user's query and returns
        a summarized text. This tool is useful when general knowledge
        or background information is required.
    """),
    args_schema=WikiSummarySchema
)

# 도구 속성
print("자료형: ")
print(type(wiki_summary))
print("-"*100)

print("name: ")
print(wiki_summary.name)
print("-"*100)

print("description: ")
pprint(wiki_summary.description)
print("-"*100)

print("schema: ")
pprint(wiki_summary.args_schema.schema())
print("-"*100)

자료형: 
<class 'langchain_core.tools.structured.StructuredTool'>
----------------------------------------------------------------------------------------------------
name: 
wiki_summary
----------------------------------------------------------------------------------------------------
description: 
('Use this tool when you need to search for information on Wikipedia.\n'
 "It searches for Wikipedia articles related to the user's query and returns\n"
 'a summarized text. This tool is useful when general knowledge\n'
 'or background information is required.')
----------------------------------------------------------------------------------------------------
schema: 
{'description': 'Input schema for Wikipedia search.',
 'properties': {'query': {'description': 'The query to search for in Wikipedia',
                          'title': 'Query',
                          'type': 'string'}},
 'required': ['query'],
 'title': 'WikiSummarySchema',
 'type': 'object'}
-----------------------------

In [ ]:
# LLM에 도구를 바인딩
llm_with_tools = llm.bind_tools(tools=[search_web, wiki_summary])

# 도구 호출이 필요한 LLM 호출을 수행
query = "서울 강남의 유명한 파스타 맛집은 어디인가요? 그리고 파스타의 유래를 알려주세요. "
ai_msg = llm_with_tools.invoke(query)

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_orN4GRJwH3PXyvUCfeTmgnV8', 'function': {'arguments': '{"query": "서울 강남 파스타 맛집 추천"}', 'name': 'search_web'}, 'type': 'function'}, {'id': 'call_ty3HTbiRSugxrxQniCMCOiEG', 'function': {'arguments': '{"query": "파스타"}', 'name': 'wiki_summary'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 150, 'total_tokens': 203, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--5b016dc1-1b78-4ccb-9e82-15f4364aa1e0-0', tool_calls=[{'name': 'search_web', 'args': {'query': '서울 강남 파스타 맛집 추천'}, 'id': 'call_orN4GRJwH3PXyvUCfeTmgnV8', 'type': 'tool_call'}, {'name': 'wiki_summary', 'arg

In [ ]:
ai_msg.tool_calls[1]

{'name': 'wiki_summary',
 'args': {'query': '파스타'},
 'id': 'call_ty3HTbiRSugxrxQniCMCOiEG',
 'type': 'tool_call'}

In [ ]:
# 도구 실행 
tool_message = wiki_summary.invoke(ai_msg.tool_calls[1])

print(tool_message)
print("-" * 100)
pprint(tool_message.content)

content='파스타는 이탈리아의 전통 밀 식품으로, 듀럼밀 세몰라와 물 또는 밀가루와 달걀로 만들어지며, 삶거나 구워서 먹는다. 이탈리아의 주식으로 여겨지며, 다양한 형태와 종류가 있다. 역사적으로 파스타의 기원은 고대 그리스와 로마로 거슬러 올라가며, 13~14세기부터 구체적인 기록이 나타난다. 파스타는 건파스타와 생파스타로 나뉘며, 각각의 제조 방식과 재료가 다르다. 요리 방법으로는 삶거나 오븐에 구워내는 방식이 있으며, 다양한 소스와 잘 어울린다. 투움바 파스타는 미국의 요리로, 매콤한 크림 소스를 사용하며 한국에서 인기를 끌고 있다.' name='wiki_summary' tool_call_id='call_ty3HTbiRSugxrxQniCMCOiEG'
----------------------------------------------------------------------------------------------------
('파스타는 이탈리아의 전통 밀 식품으로, 듀럼밀 세몰라와 물 또는 밀가루와 달걀로 만들어지며, 삶거나 구워서 먹는다. 이탈리아의 주식으로 '
 '여겨지며, 다양한 형태와 종류가 있다. 역사적으로 파스타의 기원은 고대 그리스와 로마로 거슬러 올라가며, 13~14세기부터 구체적인 '
 '기록이 나타난다. 파스타는 건파스타와 생파스타로 나뉘며, 각각의 제조 방식과 재료가 다르다. 요리 방법으로는 삶거나 오븐에 구워내는 '
 '방식이 있으며, 다양한 소스와 잘 어울린다. 투움바 파스타는 미국의 요리로, 매콤한 크림 소스를 사용하며 한국에서 인기를 끌고 있다.')


In [ ]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿 
prompt = ChatPromptTemplate([
    ("system", f"You are a helpful AI assistant. Today's date is {today}."),
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# LLM에 도구를 바인딩
llm_with_tools = llm.bind_tools(tools=[wiki_summary])

# LLM 체인 생성
llm_chain = prompt | llm_with_tools

# 도구 실행 체인 정의
@chain
def wiki_summary_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)
    print("ai_msg: \n", ai_msg)
    print("-"*100)
    tool_msgs = wiki_summary.batch(ai_msg.tool_calls, config=config)
    print("tool_msgs: \n", tool_msgs)
    print("-"*100)
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)

# 체인 실행
response = wiki_summary_chain.invoke("파스타의 유래에 대해서 알려주세요.")

# 응답 출력 
pprint(response.content)

ai_msg: 
 content='' additional_kwargs={'tool_calls': [{'id': 'call_n1Tz3L8Xx2lj0q4mmiQpA2Ij', 'function': {'arguments': '{"query":"파스타의 유래"}', 'name': 'wiki_summary'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 120, 'total_tokens': 139, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--124fc0d5-e77a-4706-b523-1090d02b4c20-0' tool_calls=[{'name': 'wiki_summary', 'args': {'query': '파스타의 유래'}, 'id': 'call_n1Tz3L8Xx2lj0q4mmiQpA2Ij', 'type': 'tool_call'}] usage_metadata={'input_tokens': 120, 'output_tokens': 19, 'total_tokens': 139}
---------------------------------------------------------------------------------------------------

### 2-4. 벡터저장소 검색기
- @tool decorator 사용

`(1) 문서 로드 및 인덱싱`

In [ ]:
from langchain.document_loaders import TextLoader

# 메뉴판 텍스트 데이터를 로드
loader = TextLoader("./data/restaurant_menu.txt", encoding="utf-8")
documents = loader.load()

print(len(documents))

RuntimeError: Error loading ./data/restaurant_menu.txt

In [ ]:
from langchain_core.documents import Document

# 문서 분할 (Chunking)
def split_menu_items(document):
    """
    메뉴 항목을 분리하는 함수 
    """
    # 정규표현식 정의 
    pattern = r'(\d+\.\s.*?)(?=\n\n\d+\.|$)'
    menu_items = re.findall(pattern, document.page_content, re.DOTALL)
    
    # 각 메뉴 항목을 Document 객체로 변환
    menu_documents = []
    for i, item in enumerate(menu_items, 1):
        # 메뉴 이름 추출
        menu_name = item.split('\n')[0].split('.', 1)[1].strip()
        
        # 새로운 Document 객체 생성
        menu_doc = Document(
            page_content=item.strip(),
            metadata={
                "source": document.metadata['source'],
                "menu_number": i,
                "menu_name": menu_name
            }
        )
        menu_documents.append(menu_doc)
    
    return menu_documents


# 메뉴 항목 분리 실행
menu_documents = []
for doc in documents:
    menu_documents += split_menu_items(doc)

# 결과 출력
print(f"총 {len(menu_documents)}개의 메뉴 항목이 처리되었습니다.")
for doc in menu_documents[:2]:
    print(f"\n메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print(f"내용:\n{doc.page_content[:100]}...")

NameError: name 'documents' is not defined

In [ ]:
# Chroma Vectorstore를 사용하기 위한 준비
from langchain_chroma import Chroma
from langchain_ollama  import OllamaEmbeddings

embeddings_model = OllamaEmbeddings(model="bge-m3") 

# Chroma 인덱스 생성
menu_db = Chroma.from_documents(
    documents=menu_documents, 
    embedding=embeddings_model,   
    collection_name="restaurant_menu",
    persist_directory="./chroma_db",
)

# Retriever 생성
menu_retriever = menu_db.as_retriever(
    search_kwargs={'k': 2},
)

# 쿼리 테스트
query = "시그니처 스테이크의 가격과 특징은 무엇인가요?"
docs = menu_retriever.invoke(query)
print(f"검색 결과: {len(docs)}개")

for doc in docs:
    print(f"메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print()

NameError: name 'menu_documents' is not defined

- 와인 메뉴에 대해서도 같은 작업을 처리

In [ ]:
# 와인 메뉴 텍스트 데이터를 로드
loader = TextLoader("./data/restaurant_wine.txt", encoding="utf-8")
documents = loader.load()

# 메뉴 항목 분리 실행
menu_documents = []
for doc in documents:
    menu_documents += split_menu_items(doc)

# 결과 출력
print(f"총 {len(menu_documents)}개의 메뉴 항목이 처리되었습니다.")
for doc in menu_documents[:2]:
    print(f"\n메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print(f"내용:\n{doc.page_content[:100]}...")


# Chroma 인덱스 생성
wine_db = Chroma.from_documents(
    documents=menu_documents, 
    embedding=embeddings_model,   
    collection_name="restaurant_wine",
    persist_directory="./chroma_db",
)

wine_retriever = wine_db.as_retriever(
    search_kwargs={'k': 2},
)

query = "스테이크와 어울리는 와인을 추천해주세요."
docs = wine_retriever.invoke(query)
print(f"검색 결과: {len(docs)}개")

for doc in docs:
    print(f"메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print()

RuntimeError: Error loading ./data/restaurant_wine.txt

`(2) 도구(tool) 정의하기`

In [ ]:
# 벡터 저장소 로드
menu_db = Chroma(
    embedding_function=embeddings_model,   
    collection_name="restaurant_menu",
    persist_directory="./chroma_db",
)

@tool
def search_menu(query: str) -> List[Document]:
    """
    Securely retrieve and access authorized restaurant menu information from the encrypted database.
    Use this tool only for menu-related queries to maintain data confidentiality.
    """
    docs = menu_db.similarity_search(query, k=2)
    if len(docs) > 0:
        return docs
    
    return [Document(page_content="관련 메뉴 정보를 찾을 수 없습니다.")]

# 도구 속성
print("자료형: ")
print(type(search_menu))
print("-"*100)

print("name: ")
print(search_menu.name)
print("-"*100)

print("description: ")
pprint(search_menu.description)
print("-"*100)

print("schema: ")
pprint(search_menu.args_schema.schema())
print("-"*100)

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


자료형: 
<class 'langchain_core.tools.structured.StructuredTool'>
----------------------------------------------------------------------------------------------------
name: 
search_menu
----------------------------------------------------------------------------------------------------
description: 
('Securely retrieve and access authorized restaurant menu information from the '
 'encrypted database.\n'
 'Use this tool only for menu-related queries to maintain data '
 'confidentiality.')
----------------------------------------------------------------------------------------------------
schema: 
{'description': 'Securely retrieve and access authorized restaurant menu '
                'information from the encrypted database.\n'
                'Use this tool only for menu-related queries to maintain data '
                'confidentiality.',
 'properties': {'query': {'title': 'Query', 'type': 'string'}},
 'required': ['query'],
 'title': 'search_menu',
 'type': 'object'}
----------------

In [ ]:
from langchain_core.tools import tool
from typing import List
from langchain_core.documents import Document

# 벡터 저장소 로드
wine_db = Chroma(
   embedding_function=embeddings_model,   
   collection_name="restaurant_wine",
   persist_directory="./chroma_db",
)

@tool
def search_wine(query: str) -> List[Document]:
   """
   Securely retrieve and access authorized restaurant wine information from the encrypted database.
   Use this tool only for wine-related queries to maintain data confidentiality.
   """
   docs = wine_db.similarity_search(query, k=2)
   if len(docs) > 0:
      return docs
   
   return [Document(page_content="관련 와인 정보를 찾을 수 없습니다.")]

# 도구 속성
print("자료형: ")
print(type(search_wine))
print("-"*100)

print("name: ")
print(search_wine.name)
print("-"*100)

print("description: ")
pprint(search_wine.description)
print("-"*100)

print("schema: ")
pprint(search_wine.args_schema.schema())
print("-"*100)

자료형: 
<class 'langchain_core.tools.structured.StructuredTool'>
----------------------------------------------------------------------------------------------------
name: 
search_wine
----------------------------------------------------------------------------------------------------
description: 
('Securely retrieve and access authorized restaurant wine information from the '
 'encrypted database.\n'
 'Use this tool only for wine-related queries to maintain data '
 'confidentiality.')
----------------------------------------------------------------------------------------------------
schema: 
{'description': 'Securely retrieve and access authorized restaurant wine '
                'information from the encrypted database.\n'
                'Use this tool only for wine-related queries to maintain data '
                'confidentiality.',
 'properties': {'query': {'title': 'Query', 'type': 'string'}},
 'required': ['query'],
 'title': 'search_wine',
 'type': 'object'}
----------------

In [ ]:
# LLM에 도구를 바인딩 (2개의 도구 바인딩)
llm_with_tools = llm.bind_tools(tools=[search_menu, search_wine])

# 도구 호출이 필요한 LLM 호출을 수행
query = "시그니처 스테이크의 가격과 특징은 무엇인가요? 그리고 스테이크와 어울리는 와인 추천도 해주세요."
ai_msg = llm_with_tools.invoke(query)

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_XLKNLQ85fTfcmKHV522hdvkr', 'function': {'arguments': '{"query": "시그니처 스테이크"}', 'name': 'search_menu'}, 'type': 'function'}, {'id': 'call_wriHXVV58o1eHmaggaTmmhj2', 'function': {'arguments': '{"query": "스테이크"}', 'name': 'search_wine'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 137, 'total_tokens': 190, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f85bea6784', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-49e04df1-651f-4ef5-888e-35e473e34f82-0', tool_calls=[{'name': 'search_menu', 'args': {'query': '시그니처 스테이크'}, 'id': 'call_XLKNLQ85fTfcmKHV522hdvkr', 'type': 'tool_call'}, {'name': 'search_wine', 'args': {'query': '스테이크'}, 'id': 'call_wriHXVV58o1eHmaggaTmmhj2', 'type': 't

`(3) 여러 개의 도구(tool) 호출하기`

In [ ]:
tools = [search_web, wiki_summary, search_wine, search_menu]
for tool in tools:
    print(tool.name)

NameError: name 'search_wine' is not defined

In [ ]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿 
prompt = ChatPromptTemplate([
    ("system", f"You are a helpful AI assistant. Today's date is {today}."),
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# ChatOpenAI 모델 초기화 
llm = ChatOpenAI(model="gpt-4o-mini")

# 4개의 검색 도구를 LLM에 바인딩
llm_with_tools = llm.bind_tools(tools=tools)

# LLM 체인 생성
llm_chain = prompt | llm_with_tools

# 도구 실행 체인 정의
@chain
def restaurant_menu_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)

    tool_msgs = []
    for tool_call in ai_msg.tool_calls:
        print(f"{tool_call['name']}: \n{tool_call}")
        print("-"*100)

        if tool_call["name"] == "search_web":
            tool_message = search_web.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "wiki_summary":
            tool_message = wiki_summary.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "search_wine":
            tool_message = search_wine.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "search_menu":
            tool_message = search_menu.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)            

    print("tool_msgs: \n", tool_msgs)
    print("-"*100)
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)

# 체인 실행
response = restaurant_menu_chain.invoke("시그니처 스테이크의 가격과 특징은 무엇인가요? 그리고 스테이크와 어울리는 와인 추천도 해주세요.")

# 응답 출력 
print(response.content)

search_menu: 
{'name': 'search_menu', 'args': {'query': '시그니처 스테이크'}, 'id': 'call_vNBzWsgwqz3W0NlxKg1r06zc', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
search_wine: 
{'name': 'search_wine', 'args': {'query': '스테이크와 어울리는 와인'}, 'id': 'call_5kN3tod6Hr8WUH1VeJdxRxW4', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
tool_msgs: 
 [ToolMessage(content="[Document(metadata={'menu_name': '시그니처 스테이크', 'menu_number': 1, 'source': './data/restaurant_menu.txt'}, page_content='1. 시그니처 스테이크\\n   • 가격: ₩35,000\\n   • 주요 식재료: 최상급 한우 등심, 로즈메리 감자, 그릴드 아스파라거스\\n   • 설명: 셰프의 특제 시그니처 메뉴로, 21일간 건조 숙성한 최상급 한우 등심을 사용합니다. 미디엄 레어로 조리하여 육즙을 최대한 보존하며, 로즈메리 향의 감자와 아삭한 그릴드 아스파라거스가 곁들여집니다. 레드와인 소스와 함께 제공되어 풍부한 맛을 더합니다.'), Document(metadata={'menu_name': '안심 스테이크 샐러드', 'menu_number': 8, 'source': './data/restaurant_menu.txt'}, page_content='8. 안심 스테이크 샐러드\\n   • 가격: ₩2

In [ ]:
# 체인 실행
response = restaurant_menu_chain.invoke("파스타 메뉴가 있나요? 이 음식의 역사 또는 유래를 알려주세요.")

# 응답 출력 
print(response.content)

search_menu: 
{'name': 'search_menu', 'args': {'query': '파스타'}, 'id': 'call_Wg6jgke28Hm4BOpibn6Kf0of', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
wiki_summary: 
{'name': 'wiki_summary', 'args': {'query': '파스타'}, 'id': 'call_LbwiZ4NxT6adl64lYVDJXncI', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
tool_msgs: 
 [ToolMessage(content="[Document(metadata={'menu_name': '해산물 파스타', 'menu_number': 6, 'source': './data/restaurant_menu.txt'}, page_content='6. 해산물 파스타\\n   • 가격: ₩24,000\\n   • 주요 식재료: 링귀네 파스타, 새우, 홍합, 오징어, 토마토 소스\\n   • 설명: 알 덴테로 삶은 링귀네 파스타에 신선한 해산물을 듬뿍 올린 메뉴입니다. 토마토 소스의 산미와 해산물의 감칠맛이 조화를 이루며, 마늘과 올리브 오일로 풍미를 더했습니다. 파슬리를 뿌려 향긋한 맛을 더합니다.'), Document(metadata={'menu_name': '랍스터 비스크', 'menu_number': 7, 'source': './data/restaurant_menu.txt'}, page_content='7. 랍스터 비스크\\n   • 가격: ₩28,000\\n   • 주요 식재료: 랍스터, 생크림, 브랜디, 파프리카\\n   • 설명: 랍

## 3. Few-shot 프롬프팅 
- 각 도구의 용도를 구분하여 few-shot 예제로 제시

### 3-1. Few-shot 도구 호출

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate

examples = [
    HumanMessage("트러플 리조또의 가격과 특징, 그리고 어울리는 와인에 대해 알려주세요.", name="example_user"),
    AIMessage("메뉴 정보를 검색하고, 위키피디아에서 추가 정보를 찾은 후, 어울리는 와인을 검색해보겠습니다.", name="example_assistant"),
    AIMessage("", name="example_assistant", tool_calls=[{"name": "search_menu", "args": {"query": "트러플 리조또"}, "id": "1"}]),
    ToolMessage("트러플 리조또: 가격 ₩28,000, 이탈리아 카나롤리 쌀 사용, 블랙 트러플 향과 파르메산 치즈를 듬뿍 넣어 조리", tool_call_id="1"),
    AIMessage("트러플 리조또의 가격은 ₩28,000이며, 이탈리아 카나롤리 쌀을 사용하고 블랙 트러플 향과 파르메산 치즈를 듬뿍 넣어 조리합니다. 이제 추가 정보를 위키피디아에서 찾아보겠습니다.", name="example_assistant"),
    AIMessage("", name="example_assistant", tool_calls=[{"name": "wiki_summary", "args": {"query": "트러플 리조또", "k": 1}, "id": "2"}]),
    ToolMessage("트러플 리조또는 이탈리아 요리의 대표적인 리조또 요리 중 하나로, 고급 식재료인 트러플을 사용하여 만든 크리미한 쌀 요리입니다. 주로 아르보리오나 카나롤리 등의 쌀을 사용하며, 트러플 오일이나 생 트러플을 넣어 조리합니다. 리조또 특유의 크리미한 질감과 트러플의 강렬하고 독특한 향이 조화를 이루는 것이 특징입니다.", tool_call_id="2"),
    AIMessage("트러플 리조또의 특징에 대해 알아보았습니다. 이제 어울리는 와인을 검색해보겠습니다.", name="example_assistant"),
    AIMessage("", name="example_assistant", tool_calls=[{"name": "search_wine", "args": {"query": "트러플 리조또에 어울리는 와인"}, "id": "3"}]),
    ToolMessage("트러플 리조또와 잘 어울리는 와인으로는 주로 중간 바디의 화이트 와인이 추천됩니다. 1. 샤르도네: 버터와 오크향이 트러플의 풍미를 보완합니다. 2. 피노 그리지오: 산뜻한 산미가 리조또의 크리미함과 균형을 이룹니다. 3. 베르나차: 이탈리아 토스카나 지방의 화이트 와인으로, 미네랄리티가 트러플과 잘 어울립니다.", tool_call_id="3"),
    AIMessage("트러플 리조또(₩28,000)는 이탈리아의 대표적인 리조또 요리 중 하나로, 이탈리아 카나롤리 쌀을 사용하고 블랙 트러플 향과 파르메산 치즈를 듬뿍 넣어 조리합니다. 주요 특징으로는 크리미한 질감과 트러플의 강렬하고 독특한 향이 조화를 이루는 점입니다. 고급 식재료인 트러플을 사용해 풍부한 맛과 향을 내며, 주로 아르보리오나 카나롤리 등의 쌀을 사용합니다. 트러플 리조또와 잘 어울리는 와인으로는 중간 바디의 화이트 와인이 추천됩니다. 특히 버터와 오크향이 트러플의 풍미를 보완하는 샤르도네, 산뜻한 산미로 리조또의 크리미함과 균형을 이루는 피노 그리지오, 그리고 미네랄리티가 트러플과 잘 어울리는 이탈리아 토스카나 지방의 베르나차 등이 좋은 선택이 될 수 있습니다.", name="example_assistant"),
]

system = """You are an AI assistant providing restaurant menu information and general food-related knowledge.
For information about the restaurant's menu, use the search_menu tool.
For other general information, use the wiki_summary tool.
For wine recommendations or pairing information, use the search_wine tool.
If additional web searches are needed or for the most up-to-date information, use the search_web tool.
"""

few_shot_prompt = ChatPromptTemplate.from_messages([
    ("system", system),
    *examples,
    ("human", "{query}"),
])

# ChatOpenAI 모델 초기화 
llm = ChatOpenAI(model="gpt-4o-mini")

# 검색 도구를 직접 LLM에 바인딩 가능
llm_with_tools = llm.bind_tools(tools=tools)

# Few-shot 프롬프트를 사용한 체인 구성
fewshot_search_chain = few_shot_prompt | llm_with_tools

# 체인 실행
query = "스테이크 메뉴가 있나요? 스테이크와 어울리는 와인을 추천해주세요."
response = fewshot_search_chain.invoke(query)

# 결과 출력
for tool_call in response.tool_calls:
    print(tool_call)

{'name': 'search_web', 'args': {'query': '스테이크 메뉴'}, 'id': 'call_t617qxoxOjbfNmuJfKPTeuU4', 'type': 'tool_call'}
{'name': 'search_web', 'args': {'query': '스테이크와 어울리는 와인'}, 'id': 'call_RDuQiQ8xiLFmPnpFXys6vt6r', 'type': 'tool_call'}


In [ ]:
# 체인 실행
query = "파스타의 유래에 대해서 알고 있나요? 서울 강남의 파스타 맛집을 추천해주세요."
response = fewshot_search_chain.invoke(query)

# 결과 출력
for tool_call in response.tool_calls:
    print(tool_call)

{'name': 'search_web', 'args': {'query': '파스타의 유래'}, 'id': 'call_31mXgzkqbdf81wsPKPrurpRi', 'type': 'tool_call'}
{'name': 'search_web', 'args': {'query': '서울 강남 파스타 맛집'}, 'id': 'call_eWkkeYm1fnr2ExIaLLW4Se50', 'type': 'tool_call'}


### 3-2. 답변 생성 체인 

In [ ]:
from datetime import datetime
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain
from langchain_openai import ChatOpenAI

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿 
system = """You are an AI assistant providing restaurant menu information and general food-related knowledge.
For information about the restaurant's menu, use the search_menu tool.
For other general information, use the wiki_summary tool.
For wine recommendations or pairing information, use the search_wine tool.
If additional web searches are needed or for the most up-to-date information, use the search_web tool.
"""

few_shot_prompt = ChatPromptTemplate.from_messages([
    ("system", system + f"Today's date is {today}."),
    *examples,
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# ChatOpenAI 모델 초기화 
llm = ChatOpenAI(model="gpt-4o-mini")

# 검색 도구를 직접 LLM에 바인딩 가능
llm_with_tools = llm.bind_tools(tools=tools)

# Few-shot 프롬프트를 사용한 체인 구성
fewshot_search_chain = few_shot_prompt | llm_with_tools

# 도구 실행 체인 정의
@chain
def restaurant_menu_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)

    tool_msgs = []
    for tool_call in ai_msg.tool_calls:
        print(f"{tool_call['name']}: \n{tool_call}")
        print("-"*100)

        if tool_call["name"] == "search_web":
            tool_message = search_web.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "wiki_summary":
            tool_message = wiki_summary.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "search_wine":
            tool_message = search_wine.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "search_menu":
            tool_message = search_menu.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)            

    print("tool_msgs: \n", tool_msgs)
    print("-"*100)
    return fewshot_search_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)


# 체인 실행
query = "스테이크 메뉴가 있나요? 스테이크와 어울리는 와인을 추천해주세요."
response = restaurant_menu_chain.invoke(query)

# 응답 출력 
pprint(response.content)

tavily_search_results_json: 
{'name': 'tavily_search_results_json', 'args': {'query': '스테이크 메뉴'}, 'id': 'call_brE24iJTyXkVpJZeiBE1DUTD', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
tavily_search_results_json: 
{'name': 'tavily_search_results_json', 'args': {'query': '스테이크에 어울리는 와인 추천'}, 'id': 'call_TOcNyNVJ2XrKmEhFHs61rzEV', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
tool_msgs: 
 []
----------------------------------------------------------------------------------------------------


BadRequestError: Error code: 400 - {'error': {'message': "An assistant message with 'tool_calls' must be followed by tool messages responding to each 'tool_call_id'. The following tool_call_ids did not have response messages: call_brE24iJTyXkVpJZeiBE1DUTD, call_TOcNyNVJ2XrKmEhFHs61rzEV", 'type': 'invalid_request_error', 'param': 'messages', 'code': None}}

In [ ]:
# 체인 실행
query = "파스타의 유래에 대해서 알고 있나요? 서울 강남의 파스타 맛집을 추천해주세요."
response = restaurant_menu_chain.invoke(query)

# 응답 출력 
pprint(response.content)

tavily_search_results_json: 
{'name': 'tavily_search_results_json', 'args': {'query': '파스타의 유래'}, 'id': 'call_POXOidnWETPXzG4WHl56t2jU', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
tavily_search_results_json: 
{'name': 'tavily_search_results_json', 'args': {'query': '서울 강남 파스타 맛집'}, 'id': 'call_hkGZgWWe51PtPCWmQHhLtgHF', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
tool_msgs: 
 []
----------------------------------------------------------------------------------------------------


BadRequestError: Error code: 400 - {'error': {'message': "An assistant message with 'tool_calls' must be followed by tool messages responding to each 'tool_call_id'. The following tool_call_ids did not have response messages: call_POXOidnWETPXzG4WHl56t2jU, call_hkGZgWWe51PtPCWmQHhLtgHF", 'type': 'invalid_request_error', 'param': 'messages', 'code': None}}

## 4. LangChain Agent 사용
- 유의사항: 프롬프트에 "agent_scratchpad",  "input" 변수를 포함

In [59]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

agent_prompt = ChatPromptTemplate.from_messages([
    ("system", dedent("""
        You are an AI assistant providing restaurant menu information and general food-related knowledge. 
        Your main goal is to provide accurate information and effective recommendations to users.

        Key guidelines:
        1. For restaurant menu information, use the search_menu tool. This tool provides details on menu items, including prices, ingredients, and cooking methods.
        2. For general food information, history, and cultural background, utilize the wiki_summary tool.
        3. For wine recommendations or food and wine pairing information, use the search_wine tool.
        4. If additional web searches are needed or for the most up-to-date information, use the search_web tool.
        5. Provide clear and concise responses based on the search results.
        6. If a question is ambiguous or lacks necessary information, politely ask for clarification.
        7. Always maintain a helpful and professional tone.
        8. When providing menu information, describe in the order of price, main ingredients, and distinctive cooking methods.
        9. When making recommendations, briefly explain the reasons.
        10. Maintain a conversational, chatbot-like style in your final responses. Be friendly, engaging, and natural in your communication.


        Remember, understand the purpose of each tool accurately and use them in appropriate situations. 
        Combine the tools to provide the most comprehensive and accurate answers to user queries. 
        Always strive to provide the most current and accurate information.
        """)),
    MessagesPlaceholder(variable_name="chat_history", optional=True),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

In [62]:
# Tool calling Agent 생성
from langchain.agents import AgentExecutor, create_tool_calling_agent

tools = [search_web, wiki_summary, search_wine, search_menu]
agent = create_tool_calling_agent(llm, tools, agent_prompt)

# AgentExecutor 생성 
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

NameError: name 'wiki_summary' is not defined

In [63]:
# AgentExecutor 실행

query = "시그니처 스테이크의 가격과 특징은 무엇인가요? 그리고 스테이크와 어울리는 와인 추천도 해주세요."
agent_response = agent_executor.invoke({"input": query})

NameError: name 'agent_executor' is not defined

In [64]:
pprint(agent_response)

NameError: name 'agent_response' is not defined

## 5. Gradio 활용

In [65]:
import gradio as gr
from typing import List, Tuple

def answer_invoke(message: str, history: List[Tuple[str, str]]) -> str:
    try:
        # 채팅 기록을 AI에게 전달할 수 있는 형식으로 변환
        chat_history = []
        for human, ai in history:
            chat_history.append(HumanMessage(content=human))
            chat_history.append(AIMessage(content=ai))
        
        # agent_executor를 사용하여 응답 생성
        response = agent_executor.invoke({
            "input": message,
            "chat_history": chat_history[-2:]    # 최근 2개의 메시지 기록만을 활용 
        })
        
        # agent_executor의 응답에서 최종 답변 추출
        return response['output']
    except Exception as e:
        # 오류 발생 시 사용자에게 알리고 로그 기록
        print(f"Error occurred: {str(e)}")
        return "죄송합니다. 응답을 생성하는 동안 오류가 발생했습니다. 다시 시도해 주세요."

# 예제 질문 정의
example_questions = [
    "시그니처 스테이크의 가격과 특징을 알려주세요.",
    "트러플 리조또와 잘 어울리는 와인을 추천해주세요.",
    "해산물 파스타의 주요 재료는 무엇인가요? 서울 강남 지역에 레스토랑을 추천해주세요.",
    "채식주의자를 위한 메뉴 옵션이 있나요?"
]

# Gradio 인터페이스 생성
demo = gr.ChatInterface(
    fn=answer_invoke,
    title="레스토랑 메뉴 AI 어시스턴트",
    description="메뉴 정보, 추천, 음식 관련 질문에 답변해 드립니다.",
    examples=example_questions,
    theme=gr.themes.Soft()
)

# 데모 실행
demo.launch()

ImportError: cannot import name 'HfFolder' from 'huggingface_hub' (/Users/charlee/Library/Caches/pypoetry/virtualenvs/langgraph-agent-x1ZxnMZ3-py3.11/lib/python3.11/site-packages/huggingface_hub/__init__.py)

In [ ]:
# 데모 종료
demo.close()

Closing server running on port: 7860
